In [176]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [177]:
# Load NEVO data
data = pd.read_csv("NEVO2023_8.0.csv", delimiter="|")

#### Rename nutrients

In [178]:
#rename nutri columns 
data.rename(columns={"Engelse naam/Food name": "food_name"}, inplace=True)
data.rename(columns={"ENERCC (kcal)": "Calories (kcal)"}, inplace=True)
data.rename(columns={"THIA (mg)": "VITB1 (mg)"}, inplace=True)
data.rename(columns={"VITA_RE (µg)": "VITA (µg)"}, inplace=True)
data.rename(columns={"CA (mg)": "Calcium (mg)"}, inplace=True)
data.rename(columns={"FE (mg)": "Iron (mg)"}, inplace=True)
data.rename(columns={"MG (mg)": "Magnesium (mg)"}, inplace=True)
data.rename(columns={"P (mg)": "Phosphorus (mg)"}, inplace=True)
data.rename(columns={"K (mg)": "Potassium (mg)"}, inplace=True)
data.rename(columns={"NA (mg)": "Sodium (mg)"}, inplace=True)
data.rename(columns={"ZN (mg)": "Zinc (mg)"}, inplace=True)

In [179]:
nutrients = [
    'FAT (g)', 'PROT (g)', 'Calories (kcal)',
    'VITB1 (mg)', 'RIBF (mg)', 'VITB6 (mg)', 'VITB12 (µg)', 'VITD (µg)',
    'VITE (mg)', 'VITC (mg)', 'VITA (µg)',
    'Calcium (mg)', 'Iron (mg)', 'Magnesium (mg)', 'Phosphorus (mg)',
    'Potassium (mg)', 'Sodium (mg)', 'Zinc (mg)'
]

data = data[nutrients + ['NEVO-code', 'Food group', 'food_name', 'Hoeveelheid/Quantity']].copy()

In [180]:
### Convert comma decimal strings to float for all nutrient columns
for col in nutrients:
    data[col] = (data[col].astype(str).str
                       .replace(',', '.', regex=False)
                       .replace('nan', None).astype(float)
    )

#### Imputing missing values

In [181]:
def impute_nutrients_knn(df, nutrient_cols, subgroup_col='Food group', n_neighbors=5):
    df_imputed = df.copy()
    
    # Create the KNN imputer
    imputer = KNNImputer(n_neighbors=n_neighbors, weights='uniform')
    
    # Process each subgroup separately
    for subgroup, subgroup_df in df.groupby(subgroup_col):
        if subgroup_df.empty:
            continue
        
        # Select nutrient values
        subgroup_nutrients = subgroup_df[nutrient_cols]
        
        # Fit and transform
        imputed_values = imputer.fit_transform(subgroup_nutrients)
        
        # Round to two decimal places
        imputed_values = np.round(imputed_values, 2)
        
        # Put the values back into the main dataframe
        df_imputed.loc[subgroup_df.index, nutrient_cols] = imputed_values
    
    return df_imputed

In [182]:
clean_data = impute_nutrients_knn(data, nutrients)

In [183]:
missing_rows = clean_data[clean_data[nutrients].isna().any(axis=1)]
print(missing_rows)

Empty DataFrame
Columns: [FAT (g), PROT (g), Calories (kcal), VITB1 (mg), RIBF (mg), VITB6 (mg), VITB12 (µg), VITD (µg), VITE (mg), VITC (mg), VITA (µg), Calcium (mg), Iron (mg), Magnesium (mg), Phosphorus (mg), Potassium (mg), Sodium (mg), Zinc (mg), NEVO-code, Food group, food_name, Hoeveelheid/Quantity]
Index: []

[0 rows x 22 columns]


#### Grouping based on the food wheel

In [184]:
# === Main Wheel of Five Group Mapping ===
wheel_mapping = {
    # Drinks
    'Non-alcoholic beverages': 'Drinks',
    'Alcoholic beverages': 'Other',

    # Vegetables and fruit
    'Vegetables': 'Vegetables and fruit',
    'Fruits': 'Vegetables and fruit',

    # Bread, Grain/Cereal Products and Potatoes
    'Cereals and cereal products': 'Bread, grain/cereal products and potatoes',
    'Bread': 'Bread, grain/cereal products and potatoes',
    'Potatoes and tubers': 'Bread, grain/cereal products and potatoes',

    # Dairy, Nuts, Fish, Legumes, Meat and Eggs
    'Milk and milk products': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Cheese': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Nuts and seeds': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Legumes': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Fish, crustacean and shellfish': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Meat and poultry': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Eggs': 'Dairy, nuts, fish, legumes, meat and eggs',
    'Meat substitutes and dairy substitutes': 'Dairy, nuts, fish, legumes, meat and eggs',

    # Spreading and Cooking Fats
    'Fats and oils': 'Spreading and cooking fats',
    'Savoury bread spreads': 'Spreading and cooking fats',

    # Outside the Wheel
    'Pastry and biscuits': 'Other',
    'Savoury snacks': 'Other',
    'Sugar, sweets and sweet sauces': 'Other',
    'Savoury sauces': 'Other',
    'Cold meat cuts': 'Other',
    'Mixed dishes': 'Other',
    'Soups': 'Other',
    'Miscellaneous foods': 'Other',
    'Foods for special nutritional use': 'Other',
    'Herbs and spices': 'Other'
}

clean_data['food_wheel_group'] = clean_data['Food group'].map(wheel_mapping)
clean_data = clean_data[clean_data['food_wheel_group'] != 'Other']

# Subgroup Mapping (matching Voedingscentrum’s Wheel of Five split)
subgroup_mapping = {
    # Vegetables and fruit
    'Vegetables': 'Vegetables',
    'Fruits': 'Fruit',

    # Dairy, nuts, fish, legumes, meat and eggs
    'Milk and milk products': 'Dairy',
    'Cheese': 'Dairy',
    'Nuts and seeds': 'Nuts',
    'Legumes': 'Fish, legumes, meat and eggs',
    'Fish, crustacean and shellfish': 'Fish, legumes, meat and eggs',
    'Meat and poultry': 'Fish, legumes, meat and eggs',
    'Eggs': 'Fish, legumes, meat and eggs',
    'Meat substitutes and dairy substitutes': 'Fish, legumes, meat and eggs'
}

clean_data['food_wheel_subgroup'] = clean_data['Food group'].map(subgroup_mapping)
clean_data['food_wheel_subgroup'] = clean_data['food_wheel_subgroup'].fillna(clean_data['food_wheel_group'])

display(clean_data)

,FAT (g),PROT (g),Calories (kcal),VITB1 (mg),RIBF (mg),VITB6 (mg),VITB12 (µg),VITD (µg),VITE (mg),VITC (mg),...,Phosphorus (mg),Potassium (mg),Sodium (mg),Zinc (mg),NEVO-code,Food group,food_name,Hoeveelheid/Quantity,food_wheel_group,food_wheel_subgroup
0,0.0,2.0,88.0,0.12,0.04,0.30,0.00,0.0,0.1,14.0,...,60.0,450.0,2.0,0.37,1,Potatoes and tubers,Potatoes raw,per 100g,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
1,0.0,2.0,88.0,0.12,0.04,0.30,0.00,0.0,0.1,25.0,...,60.0,450.0,2.0,0.20,2,Potatoes and tubers,Potatoes new raw,per 100g,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
2,0.0,2.0,88.0,0.12,0.04,0.22,0.00,0.0,0.1,5.0,...,60.0,450.0,2.0,0.30,3,Potatoes and tubers,Potatoes old raw,per 100g,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
3,1.5,12.3,356.0,0.10,0.06,0.08,0.00,0.0,0.0,0.0,...,125.0,100.0,2.0,1.50,4,Cereals and cereal products,Pasta white raw,per 100g,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
4,1.0,7.0,352.0,0.04,0.03,0.12,0.00,0.0,0.1,0.0,...,100.0,100.0,2.0,1.80,5,Cereals and cereal products,Rice white raw,per 100g,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318,4.7,17.6,128.0,0.03,0.18,0.09,0.02,0.0,1.2,0.0,...,190.0,310.0,39.0,1.30,5573,Meat substitutes and dairy substitutes,Tempeh unprepared,per 100g,"Dairy, nuts, fish, legumes, meat and eggs","Fish, legumes, meat and eggs"
2319,1.0,11.0,367.0,0.09,0.09,0.13,0.00,0.0,0.4,0.0,...,110.0,167.0,680.0,0.59,5574,Cereals and cereal products,Bakery mix for doughnuts Dutch style,per 100g,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
2320,20.4,27.3,302.0,0.03,0.27,0.06,1.81,0.3,0.7,1.0,...,606.0,72.0,661.0,3.99,5575,Cheese,Cheese 35+,per 100g,"Dairy, nuts, fish, legumes, meat and eggs",Dairy
2321,1.7,13.2,348.0,0.21,0.03,0.09,0.00,0.0,0.5,10.8,...,180.0,190.0,0.0,1.50,5576,Cereals and cereal products,Flour spelt,per 100g,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"


In [185]:
clean_data = clean_data[['NEVO-code', 'Food group', 'food_name',
                         'FAT (g)', 'PROT (g)', 'Calories (kcal)',
                         'VITB1 (mg)', 'RIBF (mg)', 'VITB6 (mg)', 'VITB12 (µg)', 'VITD (µg)',
                         'VITE (mg)', 'VITC (mg)', 'VITA (µg)', 'Calcium (mg)', 'Iron (mg)',
                         'Magnesium (mg)', 'Phosphorus (mg)', 'Potassium (mg)', 'Sodium (mg)',
                         'Zinc (mg)', 'food_wheel_group', 'food_wheel_subgroup']]

In [186]:
clean_data

,NEVO-code,Food group,food_name,FAT (g),PROT (g),Calories (kcal),VITB1 (mg),RIBF (mg),VITB6 (mg),VITB12 (µg),...,VITA (µg),Calcium (mg),Iron (mg),Magnesium (mg),Phosphorus (mg),Potassium (mg),Sodium (mg),Zinc (mg),food_wheel_group,food_wheel_subgroup
0,1,Potatoes and tubers,Potatoes raw,0.0,2.0,88.0,0.12,0.04,0.30,0.00,...,1.0,6.0,0.5,22.0,60.0,450.0,2.0,0.37,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
1,2,Potatoes and tubers,Potatoes new raw,0.0,2.0,88.0,0.12,0.04,0.30,0.00,...,0.0,6.0,0.5,14.0,60.0,450.0,2.0,0.20,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
2,3,Potatoes and tubers,Potatoes old raw,0.0,2.0,88.0,0.12,0.04,0.22,0.00,...,1.0,6.0,0.5,17.0,60.0,450.0,2.0,0.30,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
3,4,Cereals and cereal products,Pasta white raw,1.5,12.3,356.0,0.10,0.06,0.08,0.00,...,0.0,15.0,2.0,53.0,125.0,100.0,2.0,1.50,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
4,5,Cereals and cereal products,Rice white raw,1.0,7.0,352.0,0.04,0.03,0.12,0.00,...,0.0,10.0,0.4,13.0,100.0,100.0,2.0,1.80,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318,5573,Meat substitutes and dairy substitutes,Tempeh unprepared,4.7,17.6,128.0,0.03,0.18,0.09,0.02,...,0.0,88.0,1.7,60.0,190.0,310.0,39.0,1.30,"Dairy, nuts, fish, legumes, meat and eggs","Fish, legumes, meat and eggs"
2319,5574,Cereals and cereal products,Bakery mix for doughnuts Dutch style,1.0,11.0,367.0,0.09,0.09,0.13,0.00,...,0.0,22.0,1.0,21.0,110.0,167.0,680.0,0.59,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"
2320,5575,Cheese,Cheese 35+,20.4,27.3,302.0,0.03,0.27,0.06,1.81,...,200.0,959.0,0.2,36.0,606.0,72.0,661.0,3.99,"Dairy, nuts, fish, legumes, meat and eggs",Dairy
2321,5576,Cereals and cereal products,Flour spelt,1.7,13.2,348.0,0.21,0.03,0.09,0.00,...,0.0,20.0,1.6,46.0,180.0,190.0,0.0,1.50,"Bread, grain/cereal products and potatoes","Bread, grain/cereal products and potatoes"


In [187]:
clean_data.to_csv('clean_data.csv', index=False)